# Time Series Forecasting Project

This notebook demonstrates various time series forecasting techniques including:
- Exploratory Data Analysis (EDA)
- ARIMA and SARIMA models
- Prophet forecasting
- Machine Learning approaches (LSTM)
- Model evaluation and comparison

## 1. Import Required Libraries

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Time series analysis
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Deep Learning (if using LSTM)
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout

# Prophet (Facebook's forecasting tool)
# from prophet import Prophet

# Settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## 2. Load and Prepare Data

For this example, we'll use sample data. You can replace this with your own dataset.

In [ ]:
# Option 1: Load from CSV
# df = pd.read_csv('your_data.csv', parse_dates=['date_column'], index_col='date_column')

# Option 2: Generate sample data
np.random.seed(42)
date_range = pd.date_range(start='2020-01-01', end='2023-12-31', freq='D')
trend = np.linspace(100, 200, len(date_range))
seasonal = 20 * np.sin(2 * np.pi * np.arange(len(date_range)) / 365.25)
noise = np.random.normal(0, 10, len(date_range))
values = trend + seasonal + noise

df = pd.DataFrame({
    'date': date_range,
    'value': values
})
df.set_index('date', inplace=True)

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
df.head()

## 3. Exploratory Data Analysis

In [ ]:
# Basic statistics
print("Summary Statistics:")
print(df.describe())

# Check for missing values
print(f"\nMissing values: {df.isnull().sum().sum()}")

In [ ]:
# Visualize the time series
fig, axes = plt.subplots(2, 1, figsize=(15, 8))

# Original series
axes[0].plot(df.index, df['value'], linewidth=1)
axes[0].set_title('Time Series Data', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Value')
axes[0].grid(True, alpha=0.3)

# Distribution
axes[1].hist(df['value'], bins=50, edgecolor='black', alpha=0.7)
axes[1].set_title('Distribution of Values', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Value')
axes[1].set_ylabel('Frequency')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Time Series Decomposition

In [ ]:
# Decompose the time series into trend, seasonal, and residual components
decomposition = seasonal_decompose(df['value'], model='additive', period=365)

fig, axes = plt.subplots(4, 1, figsize=(15, 10))

decomposition.observed.plot(ax=axes[0], title='Observed', legend=False)
axes[0].set_ylabel('Observed')

decomposition.trend.plot(ax=axes[1], title='Trend', legend=False)
axes[1].set_ylabel('Trend')

decomposition.seasonal.plot(ax=axes[2], title='Seasonal', legend=False)
axes[2].set_ylabel('Seasonal')

decomposition.resid.plot(ax=axes[3], title='Residual', legend=False)
axes[3].set_ylabel('Residual')

plt.tight_layout()
plt.show()

## 5. Stationarity Test

Many time series models require the data to be stationary. We'll use the Augmented Dickey-Fuller test.

In [ ]:
def test_stationarity(timeseries, title='Time Series'):
    """
    Perform Augmented Dickey-Fuller test for stationarity
    """
    # Rolling statistics
    rolling_mean = timeseries.rolling(window=30).mean()
    rolling_std = timeseries.rolling(window=30).std()
    
    # Plot
    fig, ax = plt.subplots(figsize=(15, 5))
    ax.plot(timeseries, label='Original', linewidth=1)
    ax.plot(rolling_mean, label='Rolling Mean', linewidth=2)
    ax.plot(rolling_std, label='Rolling Std', linewidth=2)
    ax.legend()
    ax.set_title(f'Rolling Mean & Standard Deviation - {title}')
    ax.grid(True, alpha=0.3)
    plt.show()
    
    # ADF test
    print(f'\nAugmented Dickey-Fuller Test for {title}:')
    result = adfuller(timeseries.dropna(), autolag='AIC')
    print(f'ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value:.3f}')
    
    if result[1] <= 0.05:
        print("\n✓ Series is stationary (reject null hypothesis)")
    else:
        print("\n✗ Series is non-stationary (fail to reject null hypothesis)")
    
    return result[1] <= 0.05

test_stationarity(df['value'], 'Original Series')

In [ ]:
# If non-stationary, apply differencing
df['value_diff'] = df['value'].diff()
test_stationarity(df['value_diff'].dropna(), 'Differenced Series')

## 6. ACF and PACF Analysis

In [ ]:
# Plot ACF and PACF to determine ARIMA parameters
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

plot_acf(df['value_diff'].dropna(), lags=40, ax=axes[0])
axes[0].set_title('Autocorrelation Function (ACF)', fontsize=14, fontweight='bold')

plot_pacf(df['value_diff'].dropna(), lags=40, ax=axes[1])
axes[1].set_title('Partial Autocorrelation Function (PACF)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Train-Test Split

In [ ]:
# Split data into train and test sets (80-20 split)
train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

print(f"Training set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")
print(f"Train date range: {train_data.index.min()} to {train_data.index.max()}")
print(f"Test date range: {test_data.index.min()} to {test_data.index.max()}")

## 8. ARIMA Model

In [ ]:
# Fit ARIMA model
# ARIMA(p, d, q) where:
# p = number of autoregressive terms
# d = number of differences
# q = number of moving average terms

order = (2, 1, 2)  # Adjust based on ACF/PACF analysis
arima_model = ARIMA(train_data['value'], order=order)
arima_result = arima_model.fit()

print(arima_result.summary())

In [ ]:
# Make predictions
arima_forecast = arima_result.forecast(steps=len(test_data))

# Visualize results
plt.figure(figsize=(15, 6))
plt.plot(train_data.index, train_data['value'], label='Training Data', linewidth=1)
plt.plot(test_data.index, test_data['value'], label='Actual Test Data', linewidth=1)
plt.plot(test_data.index, arima_forecast, label='ARIMA Forecast', linewidth=2, linestyle='--')
plt.title('ARIMA Model Forecast', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate metrics
arima_mse = mean_squared_error(test_data['value'], arima_forecast)
arima_rmse = np.sqrt(arima_mse)
arima_mae = mean_absolute_error(test_data['value'], arima_forecast)

print(f"\nARIMA Model Performance:")
print(f"RMSE: {arima_rmse:.4f}")
print(f"MAE: {arima_mae:.4f}")

## 9. SARIMA Model (Seasonal ARIMA)

In [ ]:
# Fit SARIMA model
# SARIMA(p, d, q)(P, D, Q, s) where:
# (p, d, q) = non-seasonal parameters
# (P, D, Q, s) = seasonal parameters, s = seasonal period

order = (2, 1, 2)
seasonal_order = (1, 1, 1, 365)  # Daily data with yearly seasonality

sarima_model = SARIMAX(train_data['value'], 
                       order=order, 
                       seasonal_order=seasonal_order,
                       enforce_stationarity=False,
                       enforce_invertibility=False)
sarima_result = sarima_model.fit(disp=False)

print(sarima_result.summary())

In [ ]:
# Make predictions
sarima_forecast = sarima_result.forecast(steps=len(test_data))

# Visualize results
plt.figure(figsize=(15, 6))
plt.plot(train_data.index, train_data['value'], label='Training Data', linewidth=1)
plt.plot(test_data.index, test_data['value'], label='Actual Test Data', linewidth=1)
plt.plot(test_data.index, sarima_forecast, label='SARIMA Forecast', linewidth=2, linestyle='--')
plt.title('SARIMA Model Forecast', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate metrics
sarima_mse = mean_squared_error(test_data['value'], sarima_forecast)
sarima_rmse = np.sqrt(sarima_mse)
sarima_mae = mean_absolute_error(test_data['value'], sarima_forecast)

print(f"\nSARIMA Model Performance:")
print(f"RMSE: {sarima_rmse:.4f}")
print(f"MAE: {sarima_mae:.4f}")

## 10. Prophet Model (Optional)

Uncomment and install Prophet if you want to use it:
```bash
pip install prophet
```

In [ ]:
# # Prepare data for Prophet (requires 'ds' and 'y' columns)
# prophet_train = train_data.reset_index()
# prophet_train.columns = ['ds', 'y']
# 
# # Fit Prophet model
# prophet_model = Prophet(
#     yearly_seasonality=True,
#     weekly_seasonality=True,
#     daily_seasonality=False,
#     changepoint_prior_scale=0.05
# )
# prophet_model.fit(prophet_train)
# 
# # Make predictions
# future = prophet_model.make_future_dataframe(periods=len(test_data), freq='D')
# prophet_forecast = prophet_model.predict(future)
# 
# # Visualize
# fig = prophet_model.plot(prophet_forecast)
# plt.title('Prophet Model Forecast', fontsize=14, fontweight='bold')
# plt.tight_layout()
# plt.show()
# 
# # Component analysis
# fig = prophet_model.plot_components(prophet_forecast)
# plt.tight_layout()
# plt.show()
# 
# # Calculate metrics
# prophet_predictions = prophet_forecast.iloc[-len(test_data):]['yhat'].values
# prophet_rmse = np.sqrt(mean_squared_error(test_data['value'], prophet_predictions))
# prophet_mae = mean_absolute_error(test_data['value'], prophet_predictions)
# 
# print(f"\nProphet Model Performance:")
# print(f"RMSE: {prophet_rmse:.4f}")
# print(f"MAE: {prophet_mae:.4f}")

## 11. LSTM Model (Optional)

Uncomment and install TensorFlow/Keras if you want to use LSTM:
```bash
pip install tensorflow
```

In [ ]:
# # Prepare data for LSTM
# def create_sequences(data, seq_length):
#     X, y = [], []
#     for i in range(len(data) - seq_length):
#         X.append(data[i:i+seq_length])
#         y.append(data[i+seq_length])
#     return np.array(X), np.array(y)
# 
# # Scale data
# scaler = MinMaxScaler()
# scaled_data = scaler.fit_transform(df['value'].values.reshape(-1, 1))
# 
# # Create sequences
# seq_length = 30
# X, y = create_sequences(scaled_data, seq_length)
# 
# # Split into train and test
# train_size = int(len(X) * 0.8)
# X_train, X_test = X[:train_size], X[train_size:]
# y_train, y_test = y[:train_size], y[train_size:]
# 
# # Build LSTM model
# lstm_model = Sequential([
#     LSTM(50, activation='relu', return_sequences=True, input_shape=(seq_length, 1)),
#     Dropout(0.2),
#     LSTM(50, activation='relu'),
#     Dropout(0.2),
#     Dense(1)
# ])
# 
# lstm_model.compile(optimizer='adam', loss='mse')
# 
# # Train model
# history = lstm_model.fit(
#     X_train, y_train,
#     epochs=50,
#     batch_size=32,
#     validation_split=0.1,
#     verbose=1
# )
# 
# # Make predictions
# lstm_predictions = lstm_model.predict(X_test)
# lstm_predictions = scaler.inverse_transform(lstm_predictions)
# y_test_actual = scaler.inverse_transform(y_test)
# 
# # Calculate metrics
# lstm_rmse = np.sqrt(mean_squared_error(y_test_actual, lstm_predictions))
# lstm_mae = mean_absolute_error(y_test_actual, lstm_predictions)
# 
# print(f"\nLSTM Model Performance:")
# print(f"RMSE: {lstm_rmse:.4f}")
# print(f"MAE: {lstm_mae:.4f}")
# 
# # Plot results
# plt.figure(figsize=(15, 6))
# plt.plot(y_test_actual, label='Actual', linewidth=1)
# plt.plot(lstm_predictions, label='LSTM Forecast', linewidth=2, linestyle='--')
# plt.title('LSTM Model Forecast', fontsize=14, fontweight='bold')
# plt.xlabel('Time Steps')
# plt.ylabel('Value')
# plt.legend()
# plt.grid(True, alpha=0.3)
# plt.tight_layout()
# plt.show()

## 12. Model Comparison

In [ ]:
# Compare all models
results_df = pd.DataFrame({
    'Model': ['ARIMA', 'SARIMA'],
    'RMSE': [arima_rmse, sarima_rmse],
    'MAE': [arima_mae, sarima_mae]
})

# Add Prophet and LSTM if used
# results_df = results_df.append({'Model': 'Prophet', 'RMSE': prophet_rmse, 'MAE': prophet_mae}, ignore_index=True)
# results_df = results_df.append({'Model': 'LSTM', 'RMSE': lstm_rmse, 'MAE': lstm_mae}, ignore_index=True)

print("\nModel Comparison:")
print(results_df.to_string(index=False))

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

results_df.plot(x='Model', y='RMSE', kind='bar', ax=axes[0], legend=False, color='steelblue')
axes[0].set_title('RMSE Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylabel('RMSE')
axes[0].set_xlabel('')
axes[0].tick_params(axis='x', rotation=45)

results_df.plot(x='Model', y='MAE', kind='bar', ax=axes[1], legend=False, color='coral')
axes[1].set_title('MAE Comparison', fontsize=14, fontweight='bold')
axes[1].set_ylabel('MAE')
axes[1].set_xlabel('')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 13. Residual Analysis

In [ ]:
# Analyze residuals for the best model (using SARIMA as example)
residuals = test_data['value'].values - sarima_forecast.values

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Residuals over time
axes[0, 0].plot(test_data.index, residuals, linewidth=1)
axes[0, 0].axhline(y=0, color='r', linestyle='--')
axes[0, 0].set_title('Residuals Over Time', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Residual')
axes[0, 0].grid(True, alpha=0.3)

# Histogram of residuals
axes[0, 1].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Distribution of Residuals', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Residual')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].grid(True, alpha=0.3)

# Q-Q plot
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# ACF of residuals
plot_acf(residuals, lags=30, ax=axes[1, 1])
axes[1, 1].set_title('ACF of Residuals', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nResidual Statistics:")
print(f"Mean: {np.mean(residuals):.4f}")
print(f"Std Dev: {np.std(residuals):.4f}")
print(f"Min: {np.min(residuals):.4f}")
print(f"Max: {np.max(residuals):.4f}")

## 14. Future Forecasting

In [ ]:
# Make future predictions (e.g., next 90 days)
future_steps = 90

# Retrain on full dataset
final_model = SARIMAX(df['value'], 
                      order=order, 
                      seasonal_order=seasonal_order,
                      enforce_stationarity=False,
                      enforce_invertibility=False)
final_result = final_model.fit(disp=False)

# Forecast
future_forecast = final_result.forecast(steps=future_steps)
future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=future_steps, freq='D')

# Get confidence intervals
forecast_object = final_result.get_forecast(steps=future_steps)
confidence_intervals = forecast_object.conf_int()

# Visualize
plt.figure(figsize=(15, 6))
plt.plot(df.index, df['value'], label='Historical Data', linewidth=1)
plt.plot(future_dates, future_forecast, label='Future Forecast', linewidth=2, linestyle='--', color='red')
plt.fill_between(future_dates, 
                 confidence_intervals.iloc[:, 0], 
                 confidence_intervals.iloc[:, 1], 
                 alpha=0.2, color='red', label='95% Confidence Interval')
plt.title('Future Forecast with Confidence Intervals', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Display forecast table
forecast_df = pd.DataFrame({
    'Date': future_dates,
    'Forecast': future_forecast.values,
    'Lower CI': confidence_intervals.iloc[:, 0].values,
    'Upper CI': confidence_intervals.iloc[:, 1].values
})

print("\nFuture Forecast (first 10 days):")
print(forecast_df.head(10).to_string(index=False))

## 15. Conclusions and Recommendations

Based on the analysis:

1. **Model Performance**: Compare RMSE and MAE values to select the best performing model
2. **Seasonality**: The data shows clear seasonal patterns that SARIMA captures well
3. **Residuals**: Check if residuals are normally distributed and uncorrelated
4. **Future Work**: 
   - Experiment with different hyperparameters
   - Try ensemble methods combining multiple models
   - Incorporate external variables if available
   - Use cross-validation for robust evaluation

## Next Steps

1. Replace sample data with your actual time series data
2. Adjust model parameters based on your data characteristics
3. Experiment with additional models (Prophet, LSTM, etc.)
4. Fine-tune the best performing model
5. Deploy the model for production forecasting

In [ ]:
# Save the final model (optional)
# import joblib
# joblib.dump(final_result, 'time_series_model.pkl')
# print("Model saved successfully!")